In [2]:
import tensorflow as tf
# Fixes bad convolution
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [176]:
@tf.function
def solve(s, n_cycles=6, n_dims=3):
    s = tf.strings.split(s, '\n')
    s = tf.strings.bytes_split(s, 'UTF-8')
    s = s.to_tensor()
    s = tf.where(s == '.', 0.0, 1.0)
    s = s[None,...]
    s = tf.pad(s, n_dims*[2*[n_cycles]])
    s = s[None,...,None]

    kernel = tf.ones(n_dims*[3])
    kernel = tf.tensor_scatter_nd_update(kernel, [n_dims*[1]], [0])[...,None,None]
    
    @tf.function(experimental_compile=True)
    def _solve(s):
        for _ in range(n_cycles):
            active_cells = s
            convolution = tf.nn.conv3d(s, kernel, strides=[1, 1, 1, 1, 1], padding='SAME')

            cells_to_deactivate = active_cells * tf.cast(~((convolution == 2) | (convolution == 3)), tf.float32)
            cells_to_activate = (1-active_cells) * tf.cast(convolution == 3, tf.float32)

            s = active_cells - cells_to_deactivate + cells_to_activate
    
        return tf.reduce_sum(s)

    return _solve(s)
    
%timeit solve(tf.io.read_file('day17.txt'), n_cycles=6)
solve(s, n_cycles=6)

1.27 ms ± 208 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


<tf.Tensor: shape=(), dtype=float32, numpy=112.0>

In [192]:
s = tf.io.read_file('day17.txt')

@tf.function
def solve(s, n_cycles=6):
    s = tf.strings.split(s, '\n')
    s = tf.strings.bytes_split(s, 'UTF-8')
    s = s.to_tensor()
    s = tf.where(s == '.', 0.0, 1.0)
    s = s[None,None,...]
    s = tf.pad(s, 4*[2*[n_cycles]])
    s = s[None,...,None]
    
    n = 2*n_cycles + 1

    kernel = tf.ones(4*[3])
    kernel = tf.tensor_scatter_nd_update(kernel, [4*[1]], [0])[...,None,None]
    
    @tf.function(experimental_compile=True)
    def _solve(s):
        for i in range(n_cycles):
            convs = []
            # Thanks to https://github.com/funkey/conv4d/blob/master/conv4d.py#L27
            for j in range(3):
                convolution = tf.stack([
                    tf.nn.conv3d(s[:,i], kernel[j], strides=[1, 1, 1, 1, 1], padding='SAME')
                    for i in range(n)], axis=1)
                convs.append(convolution)
            
            conv0 = tf.concat((tf.zeros_like(convs[0][:,:1]), convs[0][:,:-1]), 1)
            conv1 = convs[1]
            conv2 = tf.concat((convs[2][:,1:], tf.zeros_like(convs[0][:,:1])), 1)
            
            convolution = conv0 + conv1 + conv2

            cells_to_deactivate = s * tf.cast((convolution != 2) & (convolution != 3), tf.float32)
            cells_to_activate = (1-s) * tf.cast(convolution == 3, tf.float32)

            s = s - cells_to_deactivate + cells_to_activate
    
        return tf.reduce_sum(s)

    return _solve(s)
    
%timeit solve(s, n_cycles=6)
solve(s, n_cycles=6)

15.3 ms ± 612 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


<tf.Tensor: shape=(), dtype=float32, numpy=1504.0>